# DEMO - Training and testing HerdNet on nadir aerial images

## Installations

In [1]:
# Check GPU
!nvidia-smi

Tue Apr  8 21:28:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             49W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [1]:
# Install the dependencies
!pip install albumentations
!pip install fiftyone
!pip install hydra-core
!pip install opencv-python
!pip install pandas
!pip install pillow
!pip install scikit-image
!pip install scikit-learn
!pip install scipy
!pip install wandb

In [2]:
import os
num_workers = os.cpu_count()
print(num_workers)

12


In [3]:
# Download and install the code
import sys

!git clone https://github.com/Alexandre-Delplanque/HerdNet
!cd '/content/HerdNet' && python setup.py install

sys.path.append('/content/HerdNet')

Cloning into 'HerdNet'...
remote: Enumerating objects: 570, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 570 (delta 149), reused 132 (delta 132), pack-reused 377 (from 2)
Receiving objects: 100% (570/570), 205.52 KiB | 2.63 MiB/s, done.
Resolving deltas: 100% (370/370), done.
running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_di

## Create datasets

In [13]:
# Download some of the data of Delplanque et al. (2021) as an example
!gdown 1CcTAZZJdwrBfCPJtVH6VBU3luGKIN9st -O /content/data.zip
!unzip -oq /content/data.zip -d /content

Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1CcTAZZJdwrBfCPJtVH6VBU3luGKIN9st

but Gdown can't. Please check connections and permissions.
unzip:  cannot find or open /content/data.zip, /content/data.zip.zip or /content/data.zip.ZIP.


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!cp '/content/drive/MyDrive/Colab Notebooks/trabajo_grado/data.zip' /content/

In [6]:
!unzip -oq /content/data.zip -d /content


In [7]:
# Set the seed
from animaloc.utils.seed import set_seed

set_seed(9292)

In [8]:
# Create validation patches using the patcher tool (for demo)
from animaloc.utils.useful_funcs import mkdir

mkdir('/content/data/val_patches')
!python /content/HerdNet/tools/patcher.py /content/data/val 512 512 0 /content/data/val_patches -csv /content/data/val.csv -min 0.0 -all False

Creating the buffer: 100% 111/111 [00:46<00:00,  2.40it/s]
Exporting patches: 100% 111/111 [02:11<00:00,  1.19s/it]


In [9]:
# Training, validation and test datasets
import albumentations as A

from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT

patch_size = 512
num_classes = 7
down_ratio = 2

train_dataset = CSVDataset(
    csv_file = '/content/data/train_patches.csv',
    root_dir = '/content/data/train_patches',
    albu_transforms = [
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
        A.Blur(blur_limit=15, p=0.2),
        A.Normalize(p=1.0)
        ],
    end_transforms = [MultiTransformsWrapper([
        FIDT(num_classes=num_classes, down_ratio=down_ratio),
        PointsToMask(radius=2, num_classes=num_classes, squeeze=True, down_ratio=int(patch_size//16))
        ])]
    )

val_dataset = CSVDataset(
    csv_file = '/content/data/val_patches/gt.csv',
    root_dir = '/content/data/val_patches',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

test_dataset = CSVDataset(
    csv_file = '/content/data/test.csv',
    root_dir = '/content/data/test',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

In [10]:
# Dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,  # o el que aguante la GPU
    shuffle=True,
    num_workers= num_workers,
    pin_memory=True
)


In [11]:
val_dataloader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)

In [12]:
test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

## Define HerdNet for training

In [13]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

herdnet = HerdNet(num_classes=num_classes, down_ratio=down_ratio).cuda()

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
    ]

herdnet = LossWrapper(herdnet, losses=losses)

Downloading: "http://dl.yf.io/dla/models/imagenet/dla34-ba72cf86.pth" to /root/.cache/torch/hub/checkpoints/dla34-ba72cf86.pth
100%|██████████| 60.3M/60.3M [00:04<00:00, 14.0MB/s]


## Create the Trainer

In [14]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/output'
mkdir(work_dir)

lr = 1e-4
weight_decay = 1e-3
epochs = 2

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean'
    )

evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=val_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=work_dir,
    header='validation'
    )

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=evaluator,
    work_dir=work_dir
    )

# Solo por si toca vaciar memoria de GPU

In [21]:
import torch
torch.cuda.empty_cache()

In [23]:
import gc
gc.collect()
torch.cuda.empty_cache()

## Start training

In [24]:
trainer.start(warmup_iters=100, checkpoints='best', select='max', validate_on='f1_score')

INFO:20250408_training:[TRAINING] - Epoch: [1] [  1/389] eta: 0:20:39 lr: 0.000002 loss: 20.8399 (20.8399) focal_loss: 19.6296 (19.6296) ce_loss: 1.2103 (1.2103) time: 3.1874 data: 1.7886 max mem: 14747


[TRAINING] - Epoch: [1] [  1/389] eta: 0:20:39 lr: 0.000002 loss: 20.8399 (20.8399) focal_loss: 19.6296 (19.6296) ce_loss: 1.2103 (1.2103) time: 3.1874 data: 1.7886 max mem: 14747


INFO:20250408_training:[TRAINING] - Epoch: [1] [ 51/389] eta: 0:07:35 lr: 0.000051 loss: 9.6175 (14.7240) focal_loss: 8.4885 (13.5756) ce_loss: 0.9962 (1.1483) time: 1.2795 data: 0.0010 max mem: 14747


[TRAINING] - Epoch: [1] [ 51/389] eta: 0:07:35 lr: 0.000051 loss: 9.6175 (14.7240) focal_loss: 8.4885 (13.5756) ce_loss: 0.9962 (1.1483) time: 1.2795 data: 0.0010 max mem: 14747


INFO:20250408_training:[TRAINING] - Epoch: [1] [101/389] eta: 0:06:20 lr: 0.000100 loss: 3.1990 (9.5922) focal_loss: 2.4551 (8.5731) ce_loss: 0.7300 (1.0192) time: 1.2981 data: 0.0019 max mem: 14747


...

/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transf

[VALIDATION] - Epoch: [2] [521/532] eta: 0:00:01 n: 1 recall: 1.0 precision: 0.2 f1-score: 0.33 MAE: 4.0 MSE: 16.0 RMSE: 4.0 time: 0.1444 data: 0.0118 max mem: 14747


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transf

[VALIDATION] - Epoch: [2] [531/532] eta: 0:00:00 n: 1 recall: 0.0 precision: 0.0 f1-score: 0.0 MAE: 1.0 MSE: 1.0 RMSE: 1.0 time: 0.1537 data: 0.0139 max mem: 14747
[VALIDATION] - Epoch: [2] [532/532] eta: 0:00:00 n: 2 recall: 1.0 precision: 0.11 f1-score: 0.19 MAE: 17.0 MSE: 289.0 RMSE: 17.0 time: 0.1532 data: 0.0135 max mem: 14747
[VALIDATION] - Epoch: [2] Total time: 0:01:05 (0.1224 s / it)
[VALIDATION] - Epoch: [2] f1_score: 0.5138
Best model saved - Epoch 2 - Validation value: 0.513839


LossWrapper(
  (model): HerdNet(
    (base_0): DLA(
      (base_layer): Sequential(
        (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level0): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level1): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level2): Tree(
        (tree1): BasicBlock(
          (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-0

## Test the model

In [15]:
# Path to your .pth file
import gdown

pth_path = ''

if not pth_path:
    gdown.download(
        'https://drive.google.com/uc?export=download&id=1-WUnBC4BJMVkNvRqalF_HzA1_pRkQTI_',
        '/content/20220413_herdnet_model.pth'
        )
    pth_path = '/content/20220413_herdnet_model.pth'

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1-WUnBC4BJMVkNvRqalF_HzA1_pRkQTI_
From (redirected): https://drive.google.com/uc?export=download&id=1-WUnBC4BJMVkNvRqalF_HzA1_pRkQTI_&confirm=t&uuid=1ab40443-9cb8-4f80-a2dd-aacdb2e38fab
To: /content/20220413_herdnet_model.pth
100%|██████████| 220M/220M [00:02<00:00, 98.6MB/s]


In [16]:
# Create output folder
test_dir = '/content/test_output'
mkdir(test_dir)

In [17]:
# Create output folder
import os
test_dir = '/content/test_output'
os.makedirs(test_dir, exist_ok=True) # Use os.makedirs instead of mkdir

In [18]:
# Load trained parameters
from animaloc.models import load_model

herdnet = load_model(herdnet, pth_path=pth_path)

In [19]:
# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=test_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
    )

In [20]:
# Start testing
test_f1_score = test_evaluator.evaluate(returns='f1_score')

/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
INFO:20250408_evaluation:test [  1/258] eta: 0:22:23 n: 1 recall: 1.0 precision: 0.2 f1-score: 0.33 MAE: 4.0 MSE: 16.0 RMSE: 4.0 time: 5.2082 data: 0.7421 max mem: 1471


test [  1/258] eta: 0:22:23 n: 1 recall: 1.0 precision: 0.2 f1-score: 0.33 MAE: 4.0 MSE: 16.0 RMSE: 4.0 time: 5.2082 data: 0.7421 max mem: 1471


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transf

test [ 11/258] eta: 0:15:04 n: 1 recall: 1.0 precision: 1.0 f1-score: 1.0 MAE: 0.0 MSE: 0.0 RMSE: 0.0 time: 3.6472 data: 0.6522 max mem: 1471


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transf

...

test [241/258] eta: 0:00:59 n: 13 recall: 0.46 precision: 0.86 f1-score: 0.6 MAE: 6.0 MSE: 36.0 RMSE: 6.0 time: 2.9884 data: 0.5440 max mem: 1474


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transf

test [251/258] eta: 0:00:26 n: 2 recall: 1.0 precision: 1.0 f1-score: 1.0 MAE: 0.0 MSE: 0.0 RMSE: 0.0 time: 3.0137 data: 0.5354 max mem: 1474


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:250: UserWarning: Got processor for keypoints, but no transf

test [258/258] eta: 0:00:03 n: 3 recall: 0.67 precision: 0.67 f1-score: 0.67 MAE: 0.0 MSE: 0.0 RMSE: 0.0 time: 3.0080 data: 0.5385 max mem: 1474
test Total time: 0:14:10 (3.2952 s / it)


In [21]:
# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

F1 score = 83%


In [22]:
# Get the detections
detections = test_evaluator.results
detections

,class,n,recall,precision,f1_score,confusion,mae,mse,rmse,ap
0,1,675,0.875556,0.838298,0.856522,0.057416,1.718310,7.887324,2.808438,0.801938
1,2,349,0.727794,0.916968,0.811502,0.066176,3.500000,35.642857,5.970164,0.693974
2,3,477,0.903564,0.848425,0.875127,0.092632,1.028302,3.952830,1.988173,0.859655
3,4,74,0.445946,0.388235,0.415094,0.057143,2.243243,9.810811,3.132221,0.272273
4,5,36,0.694444,0.657895,0.675676,0.193548,1.230769,2.153846,1.467599,0.595822
5,6,688,0.728198,0.675202,0.700699,0.001992,2.601942,19.669903,4.435076,0.655523
6,binary,2299,0.844715,0.824628,0.834551,0.000000,1.906977,13.542636,3.680032,0.722731
